In [61]:
import pandas as pd
import numpy as np
import warnings

from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [62]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/processed.csv')
parameters = pd.read_csv(path+'parameters/neutralnetworkclassifier.csv')

In [63]:
N = 2021

In [64]:
params_to_drop = ['season', 'round', 'podium', 'driver_points_from']

In [65]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < N]
X_train = train.drop(params_to_drop, axis=1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [66]:
data.columns

Index(['season', 'round', 'podium', 'q_delta', 'starting_grid',
       'driver_points_before', 'constructor_points_before',
       'driver_points_from', 'driver_points_per', 'points_percentage',
       'driver_last_3', 'constructor_last_3', 'stage_q1', 'stage_q2',
       'stage_q3'],
      dtype='object')

In [67]:
def get_predictions(model):
    predictions = []
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.podium
        
        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
        
        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns=['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df['predicted'] = prediction_df.proba_1.map(lambda x: 1 if float(x) == float(prediction_df.proba_1.max()) else 0)
        

        predictions += list(prediction_df.predicted.values)

    return predictions

In [68]:
parameters.head()

,Unnamed: 0,model,hidden_layer_sizes,activation,solver,alpha,score
0,210,neural_network_classifier,"(80, 20, 40, 5)",relu,sgd,0.143845,0.590909
1,200,neural_network_classifier,"(80, 20, 40, 5)",relu,sgd,0.000100,0.590909
2,209,neural_network_classifier,"(80, 20, 40, 5)",relu,sgd,0.069519,0.590909
3,208,neural_network_classifier,"(80, 20, 40, 5)",relu,sgd,0.033598,0.590909
4,207,neural_network_classifier,"(80, 20, 40, 5)",relu,sgd,0.016238,0.590909


In [69]:
params = parameters.iloc[1]
hidden_layer_sizes = eval(params.hidden_layer_sizes)
activation = params.activation
solver = params.solver
alpha = params.alpha

params

Unnamed: 0                                  200
model                 neural_network_classifier
hidden_layer_sizes              (80, 20, 40, 5)
activation                                 relu
solver                                      sgd
alpha                                    0.0001
score                                  0.590909
Name: 1, dtype: object

In [70]:
model = MLPClassifier(
    hidden_layer_sizes=hidden_layer_sizes,
    activation=activation, 
    solver=solver, alpha=alpha, 
    random_state=1)

model.fit(X_train, y_train)

predictions = get_predictions(model)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [71]:
test = merged.query('season ==@N')
test['predicted'] = predictions
test = test[['season', 'round', 'circuit_id', 'driver', 'starting_grid' ,'podium', 'predicted']]

In [72]:
test.query('predicted == 1')

,season,round,circuit_id,driver,starting_grid,podium,predicted
2743,2021,1,bahrain,max_verstappen,1,2,1
2763,2021,2,imola,lewis_hamilton,1,2,1
2781,2021,3,portimao,lewis_hamilton,2,1,1
2801,2021,4,catalunya,lewis_hamilton,1,1,1
2821,2021,5,monaco,max_verstappen,2,1,1
2853,2021,6,baku,lewis_hamilton,2,15,1
2858,2021,7,ricard,max_verstappen,1,1,1
2876,2021,8,red_bull_ring,max_verstappen,1,1,1
2896,2021,9,red_bull_ring,max_verstappen,1,1,1
2917,2021,11,hungaroring,lewis_hamilton,1,2,1
